# What is the bootstrapping approach for the task learning?
In the context of teaching, bootstrapping approach often involve starting with basic knowledge or skills and then iteratively improving or expanding upon them through additional learning experiences, in this case by interacting with a human mentor. This approach allows learners to gradually acquire more complex knowledge or abilities by building on what they already know. Bootstrapping is a common strategy in education and learning systems, helping individuals incrementally develop their expertise.

For the learning of the pouring task, a task learning strategy has been developed by leveraging primitive actions that the PR2 robot can execute using PyCRAM action and/or motion designators. This strategy involves breaking down the pouring task into simpler, fundamental actions that the robot can understand and execute. By utilizing these primitive actions, the robot can gradually learn and perform the complete pouring task. When the instruction to save the learning plan is given, the robot can then incorporate this complex plan into its existing repertoire of skills. This method allows the robot to accumulate and store knowledge about various tasks and their constraints, contributing to its overall capabilities.

# List of bootstrapping instructions


## Start teaching 

## At first Initialize an environment and robot agent

In [2]:
from pycram.designators.action_designator import PickUpAction, PlaceAction, ParkArmsAction, MoveTorsoAction, \
    NavigateAction, PourAction
from pycram.designators.object_designator import *
from pycram.process_module import simulated_robot
from pycram.enums import Arms
from pycram.pose import Pose
# from pycram.designator import ObjectDesignatorDescription as odd
from pycram.bullet_world import BulletWorld, Object
from pycram.designators.location_designator import CostmapLocation, SemanticCostmapLocation
import tf




## Pour
Pour action designator lets the robot pour a particle(not yet simulated) from one container to the other. It is an extention of pick and place example given earlier. 

This action designator only works in the apartment environment for the moment. 

In [4]:
from pycram.designators.action_designator import PickUpAction, PlaceAction, ParkArmsAction, MoveTorsoAction, \
    NavigateAction, PourAction
from pycram.designators.object_designator import *
from pycram.process_module import simulated_robot
from pycram.enums import Arms
from pycram.pose import Pose
# from pycram.designator import ObjectDesignatorDescription as odd
from pycram.bullet_world import BulletWorld, Object
from pycram.designators.location_designator import CostmapLocation, SemanticCostmapLocation
import tf


world = BulletWorld()
world.set_gravity([0, 0, -9.8])


# source container
milk = Object("milk", "milk", "milk.stl", pose=Pose([2.4, 2.5, 1]))
# destination container
bowl = Object("bowl", "bowl", "bowl.stl", pose=Pose([2.4, 2.8, 0.98]))
pr2 = Object("pr2", "robot", "pr2.urdf", pose=Pose([1.2, 2.5, 0]))

pr2_desig = BelieveObject(names=["pr2"])
milk_desig = BelieveObject(names=["milk"])
bowl_desig = BelieveObject(names=["bowl"])

apartment = Object("apartment", "environment", "apartment.urdf")

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/abhijit/catkin_ws/src/pycram/src/pycram/bullet_world.py", line 517, in run
    self.world.client_id = p.connect(p.GUI)
pybullet.error: Only one local in-process GUI/GUI_SERVER connection allowed. Use DIRECT connection mode or start a separate GUI physics server (ExampleBrowser, App_SharedMemoryPhysics_GUI, App_SharedMemoryPhysics_VR) and connect over SHARED_MEMORY, UDP or TCP instead.


error: Not connected to physics server.

In [ ]:
## Provide pouring plan for robot to follow by using bootstrapping instructions

In [3]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.3]).resolve().perform()

    # find pick up pose for source container
    pickup_pose = CostmapLocation(target=milk_desig.resolve(), reachable_for=pr2_desig.resolve()).resolve()
    pickup_arm = pickup_pose.reachable_arms[0]

    # navigate to the source container
    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    # perform pickup action
    PickUpAction(object_designator_description=milk_desig, arms=[pickup_arm], grasps=["front"]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
    # calculate pouring angle in quaternion
    quaternion = tf.transformations.quaternion_from_euler(90, 0, 0, axes="sxyz")
    # for simplicity calculate tilting pose where the destination container is. In this case a bowl
    tilting_pose = SemanticCostmapLocation.Location(pose=Pose( bowl.original_pose.position_as_list(), list(quaternion)))
    revert_tilting_pose = SemanticCostmapLocation.Location(pose=Pose(bowl.original_pose.position_as_list(), [0.0, 0, 0, 1]))
    # do pouring by tilting, and accept time interval
    PourAction(milk_desig, pouring_location=[tilting_pose.pose], revert_location=[revert_tilting_pose.pose],
               arms=[pickup_arm], wait_duration=5).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
    # place the source container to its original position
    place_pose = SemanticCostmapLocation.Location(pose=Pose(milk.original_pose.position_as_list(), [0.0, 0, 0, 1.0]))
    PlaceAction(milk_desig, target_locations=[place_pose.pose], arms=[pickup_arm]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
    # move pr2 to end pose
    end_pose = SemanticCostmapLocation.Location(pose=Pose([1, 2.5, 0], [0.0, 0, 0, 1.0]))
    NavigateAction(target_locations=[end_pose.pose]).resolve().perform()

AttributeError: 'NoneType' object has no attribute 'get_pose'

In [4]:
world.reset_bullet_world()

NameError: name 'world' is not defined